<a id='index'></a>

# More Data Proccesing with Pandas

## Índice

- [Merging DataFrames](#merging)

    1. [merge](#c_merge)
        - [how=](#c_how)
        - On
            - [Single Column](#c_on_single_col)
            - [Multiple Column](#c_on_mul_col)
    1. [concat](#c_concat)

---

- [Pandas Idioms](#idioms)
    1. [](#c_)
    1. [](#c_)
    1. [](#c_)
    1. [](#c_)

<a id='merging'></a>

[Índice](#index)

## Merging DataFrames

---

In [1]:
import pandas as pd

In [11]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                         {'Name': 'Sally', 'Role': 'Course liasion'},
                         {'Name': 'James', 'Role': 'Grader'}])
staff_df.set_index('Name', inplace=True)

student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                           {'Name': 'Mike', 'School': 'Law'},
                           {'Name': 'Sally', 'School': 'Engineering'}])
student_df.set_index('Name', inplace=True)
print(staff_df)
print(student_df)

                 Role
Name                 
Kelly  Director of HR
Sally  Course liasion
James          Grader
            School
Name              
James     Business
Mike           Law
Sally  Engineering


<a id='c_merge'></a>

[Índice](#index)

### `merge`

<a id='c_how'></a>

[Índice](#index)

#### `how=`

- outer
- inner
- left
- right

In [14]:
pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course liasion,Engineering


In [15]:
pd.merge(staff_df, student_df, how='inner', left_index=True, right_index=True)

,Role,School
Name,,
Sally,Course liasion,Engineering
James,Grader,Business


In [16]:
pd.merge(staff_df, student_df, how='left', left_index=True, right_index=True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course liasion,Engineering
James,Grader,Business


In [17]:
pd.merge(staff_df, student_df, how='right', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


In [21]:
staff_df.reset_index(), student_df.reset_index()

(    Name            Role
 0  Kelly  Director of HR
 1  Sally  Course liasion
 2  James          Grader,
     Name       School
 0  James     Business
 1   Mike          Law
 2  Sally  Engineering)

<a id='c_on_single_col'></a>

[Índice](#index)

#### `on=` Single Column

In [22]:
pd.merge(staff_df, student_df, how='right', on='Name')

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course liasion,Engineering


In [24]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR','Location': 'State Street'},
                         {'Name': 'Sally', 'Role': 'Course liasion', 'Location': 'Washington Avenue'},
                         {'Name': 'James', 'Role': 'Grader', 'Location': 'Washington Avenue'}])

student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 'Location': '1024 Billiard Avenue'},
                           {'Name': 'Mike', 'School': 'Law', 'Location': 'Fraternity House #22'},
                           {'Name': 'Sally', 'School': 'Engineering', 'Location': '512 Wilson Crescent'}])

In [25]:
pd.merge(staff_df, student_df, how='left', on='Name')

,Name,Role,Location_x,School,Location_y
0,Kelly,Director of HR,State Street,NaN,NaN
1,Sally,Course liasion,Washington Avenue,Engineering,512 Wilson Crescent
2,James,Grader,Washington Avenue,Business,1024 Billiard Avenue


In [29]:
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name' : 'Desjardins',
                          'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name' : 'Brooks',
                          'Role': 'Course liasion'},
                         {'First Name': 'James', 'Last Name' : 'Wilde',
                          'Role': 'Grader'}])

student_df = pd.DataFrame([{'First Name': 'James', 'Last Name' : 'Hammond',
                            'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name' : 'Smith',
                            'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name' : 'Brooks',
                            'School': 'Engineering'}])

<a id='c_on_mul_col'></a>

[Índice](#index)

#### `on=` Multiple Columns

In [30]:
pd.merge(staff_df, student_df, how='inner', on=['First Name', 'Last Name'])

,First Name,Last Name,Role,School
0,Sally,Brooks,Course liasion,Engineering


In [31]:
%%capture

# to let ignore the bad lines on the csv, and supress the warning messages
df_2011 = pd.read_csv('datasets/MERGED2011_12_PP.csv', error_bad_lines=False)
df_2012 = pd.read_csv('datasets/MERGED2012_13_PP.csv', error_bad_lines=False)
df_2013 = pd.read_csv('datasets/MERGED2012_13_PP.csv', error_bad_lines=False)

In [32]:
df_2011.head()

,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,OMAWDP8_NOTFIRSTTIME_POOLED_SUPP,OMENRUP_NOTFIRSTTIME_POOLED_SUPP,OMENRYP_FULLTIME_POOLED_SUPP,OMENRAP_FULLTIME_POOLED_SUPP,OMAWDP8_FULLTIME_POOLED_SUPP,OMENRUP_FULLTIME_POOLED_SUPP,OMENRYP_PARTTIME_POOLED_SUPP,OMENRAP_PARTTIME_POOLED_SUPP,OMAWDP8_PARTTIME_POOLED_SUPP,OMENRUP_PARTTIME_POOLED_SUPP
0,100654.0,100200.0,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100663.0,105200.0,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100690.0,2503400.0,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100706.0,105500.0,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100724.0,100500.0,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
df_2011.info(), df_2012.info(), df_2013.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15235 entries, 0 to 15234
Columns: 1977 entries, UNITID to OMENRUP_PARTTIME_POOLED_SUPP
dtypes: float64(197), object(1780)
memory usage: 229.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7793 entries, 0 to 7792
Columns: 1977 entries, UNITID to OMENRUP_PARTTIME_POOLED_SUPP
dtypes: float64(743), int64(11), object(1223)
memory usage: 117.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7793 entries, 0 to 7792
Columns: 1977 entries, UNITID to OMENRUP_PARTTIME_POOLED_SUPP
dtypes: float64(743), int64(11), object(1223)
memory usage: 117.5+ MB


(None, None, None)

In [37]:
len(df_2011), len(df_2012), len(df_2013)

(15235, 7793, 7793)

<a id='c_concat'></a>

[Índice](#index)

### `concat`

In [39]:
frames = [df_2011, df_2012, df_2013]
pd.concat(frames).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30821 entries, 0 to 7792
Columns: 1977 entries, UNITID to OMENRUP_PARTTIME_POOLED_SUPP
dtypes: float64(197), object(1780)
memory usage: 465.1+ MB


In [41]:
rows = 0
for i in frames:
    rows += len(i)
rows

30821

In [42]:
pd.concat(frames, keys=['2011', '2012', '2013'])

UNITID       OPEID OPEID6  \
2011 0       100654.0    100200.0   1002   
     1       100663.0    105200.0   1052   
     2       100690.0   2503400.0  25034   
     3       100706.0    105500.0   1055   
     4       100724.0    100500.0   1005   
...               ...         ...    ...   
2013 7788  47691101.0   4205801.0  42058   
     7789  47701101.0  10145905.0   1459   
     7790  47702001.0  10145903.0   1459   
     7791  47702002.0  10145902.0   1459   
     7792  47702003.0  10145906.0   1459   

                                              INSTNM           CITY STABBR  \
2011 0                      Alabama A & M University         Normal     AL   
     1           University of Alabama at Birmingham     Birmingham     AL   
     2                            Amridge University     Montgomery     AL   
     3           University of Alabama in Huntsville     Huntsville     AL   
     4                      Alabama State University     Montgomery     AL   
...                                              ...            ...    ...   
2013 7788  SAE Institute of Technology San Francisco     Emeryville     CA   
     7789      Strayer University-Bloomington Campus    Bloomington     MN   
     7790       Strayer University-Schaumburg Campus     Schaumburg     IL   
     7791    Strayer University-Downers Grove Campus  Downers Grove     IL   
     7792           Strayer University-Aurora Campus         Aurora     IL   

                  ZIP  ACCREDAGENCY INSTURL NPCURL  ...  \
2011 0          35762           NaN     NaN    NaN  ...   
     1     35294-0110           NaN     NaN    NaN  ...   
     2     36117-3553           NaN     NaN    NaN  ...   
     3          35899           NaN     NaN    NaN  ...   
     4     36104-0271           NaN     NaN    NaN  ...   
...               ...           ...     ...    ...  ...   
2013 7788       94608           NaN     NaN    NaN  ...   
     7789   554311411           NaN     NaN    NaN  ...   
     7790   601735081           NaN     NaN    NaN  ...   
     7791   605151169           NaN     NaN    NaN  ...   
     7792   605066220           NaN     NaN    NaN  ...   

          OMAWDP8_NOTFIRSTTIME_POOLED_SUPP OMENRUP_NOTFIRSTTIME_POOLED_SUPP  \
2011 0                                 NaN                              NaN   
     1                                 NaN                              NaN   
     2                                 NaN                              NaN   
     3                                 NaN                              NaN   
     4                                 NaN                              NaN   
...                                    ...                              ...   
2013 7788                              NaN                              NaN   
     7789                              NaN                              NaN   
     7790                              NaN                              NaN   
     7791                              NaN                              NaN   
     7792                              NaN                              NaN   

          OMENRYP_FULLTIME_POOLED_SUPP OMENRAP_FULLTIME_POOLED_SUPP  \
2011 0                             NaN                          NaN   
     1                             NaN                          NaN   
     2                             NaN                          NaN   
     3                             NaN                          NaN   
     4                             NaN                          NaN   
...                                ...                          ...   
2013 7788                          NaN                          NaN   
     7789                          NaN                          NaN   
     7790                          NaN                          NaN   
     7791                          NaN                          NaN   
     7792                          NaN                          NaN   

          OMAWDP8_FULLTIME_POOLED_SUPP

<a id='idioms'></a>

[Índice](#index)

## Pandas Idioms

---

In [1]:
import pandas as pd
import numpy as np

import timeit

In [ ]:
df = pd.read_csv('datasets/')